In [5]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

from sklearn.linear_model import LogisticRegression

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [ ]:
q = """
WITH zip AS (
    SELECT
        purch_client_crm_id,
        zip AS zip_code
    FROM
        custom.cth_v_ticket_2425
    LEFT JOIN
        custom.seatgeek_v_clients ON cth_v_ticket_2425.purch_client_crm_id = seatgeek_v_clients.crm_id
    LEFT JOIN
        custom.golden_record_v_addresses ON seatgeek_v_clients.email = golden_record_v_addresses.email
    WHERE
        ticket_type IN ('Annual Suites','Full','Premier')
    GROUP BY
        purch_client_crm_id,
        zip
),
touchpoints AS (
    SELECT DISTINCT
        a.sf_contactid
    FROM
        custom.korepss_contacts a
    WHERE
        a.contacttype = 'Member'
),
touchpoints_agg AS (
    SELECT
        a.primary_ticketing_id,
        d.last_activity_date_new__c,
        COUNT(DISTINCT act.activityid) AS touchpoints
    FROM
        custom.korepss_v_contacts a
        LEFT JOIN custom.korepss_contacts b ON a.sf_contactid = b.sf_contactid
        LEFT JOIN custom.korepss_accounts_extension d ON a.sf_accountid = d.sf_accountid
        LEFT JOIN custom.korepss_v_users c ON b.ticketing_service_rep = c.sf_userid
        LEFT JOIN custom.korepss_v_activities act ON a.sf_contactid = act.contactid AND b.ticketing_service_rep = act.ownerid
    WHERE
        a.sf_contactid IN (SELECT sf_contactid FROM touchpoints)
        AND act.completed_on_coalesce >= '7/1/2024'
    GROUP BY
        a.primary_ticketing_id,
        d.last_activity_date_new__c
),
tenure AS (
    SELECT
        purchaser_ticketing_id,
        LISTAGG(DISTINCT season),
        CASE
            WHEN LISTAGG(DISTINCT season) LIKE '2021-222022-232023-242024-25' THEN 4
            WHEN LISTAGG(DISTINCT season) LIKE '2022-232023-242024-25' THEN 3
            WHEN LISTAGG(DISTINCT season) LIKE '2023-242024-25' THEN 2
            WHEN LISTAGG(DISTINCT season) LIKE '%2024-25%' THEN 1
            ELSE 0
        END AS tenure
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Annual Suites','Full','Premier')
    GROUP BY
        purchaser_ticketing_id
),
attendance_info AS (
    SELECT
        ticketing_id,
        num_games_attended,
        self_show_rate,
        seat_show_rate,
        real_show_rate
    FROM
        custom.cth_v_show_rates
    WHERE
        season = '2024-25'
),
total_spend AS (
    SELECT
        purchaser_ticketing_id,
        SUM(gross_revenue) AS gross_revenue
    FROM
        custom.cth_v_historical_ticket
    WHERE
        season = '2024-25'
    GROUP BY
        purchaser_ticketing_id
),
location_ticket_type AS (
    SELECT
        purchaser_ticketing_id,
        LISTAGG(DISTINCT ticket_type, ', ') AS ticket_types,
        LISTAGG(DISTINCT arena_level_internal, ', ') AS arena_level_internals
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Annual Suites','Full','Premier')
        AND season = '2024-25'
    GROUP BY
        purchaser_ticketing_id
)
SELECT
    zip.purch_client_crm_id,
    zip.zip_code,
    touchpoints_agg.touchpoints,
    tenure.tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    total_spend.gross_revenue,
    location_ticket_type.ticket_types,
    location_ticket_type.arena_level_internals
FROM
    zip
LEFT JOIN
    touchpoints_agg ON zip.purch_client_crm_id = touchpoints_agg.primary_ticketing_id
LEFT JOIN
    tenure ON zip.purch_client_crm_id = tenure.purchaser_ticketing_id
LEFT JOIN
    attendance_info ON zip.purch_client_crm_id = attendance_info.ticketing_id
LEFT JOIN
    total_spend ON zip.purch_client_crm_id = total_spend.purchaser_ticketing_id
LEFT JOIN
    location_ticket_type ON zip.purch_client_crm_id = location_ticket_type.purchaser_ticketing_id
"""

df_2425 = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [7]:
q = """
WITH zip AS (
    SELECT
        purch_client_crm_id,
        zip AS zip_code
    FROM
        custom.cth_v_ticket_2324
    LEFT JOIN
        custom.seatgeek_v_clients ON cth_v_ticket_2324.purch_client_crm_id = seatgeek_v_clients.crm_id
    LEFT JOIN
        custom.golden_record_v_addresses ON seatgeek_v_clients.email = golden_record_v_addresses.email
    WHERE
        ticket_type IN ('Annual Suites','Full','Premier')
    GROUP BY
        purch_client_crm_id,
        zip
),
touchpoints AS (
    SELECT DISTINCT
        a.sf_contactid
    FROM
        custom.korepss_contacts a
    WHERE
        a.contacttype = 'Member'
),
touchpoints_agg AS (
    SELECT
        a.primary_ticketing_id,
        d.last_activity_date_new__c,
        COUNT(DISTINCT act.activityid) AS touchpoints
    FROM
        custom.korepss_v_contacts a
        LEFT JOIN custom.korepss_contacts b ON a.sf_contactid = b.sf_contactid
        LEFT JOIN custom.korepss_accounts_extension d ON a.sf_accountid = d.sf_accountid
        LEFT JOIN custom.korepss_v_users c ON b.ticketing_service_rep = c.sf_userid
        LEFT JOIN custom.korepss_v_activities act ON a.sf_contactid = act.contactid AND b.ticketing_service_rep = act.ownerid
    WHERE
        a.sf_contactid IN (SELECT sf_contactid FROM touchpoints)
        AND act.completed_on_coalesce >= '7/1/2023'
        AND act.completed_on_coalesce <= '7/1/2024'
    GROUP BY
        a.primary_ticketing_id,
        d.last_activity_date_new__c
),
tenure AS (
    SELECT
        purchaser_ticketing_id,
        LISTAGG(DISTINCT season),
        CASE
            WHEN LISTAGG(DISTINCT season) LIKE '2021-222022-232023-24' THEN 3
            WHEN LISTAGG(DISTINCT season) LIKE '2022-232023-24' THEN 2
            WHEN LISTAGG(DISTINCT season) LIKE '%2023-24%' THEN 1
            ELSE 0
        END AS tenure
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Annual Suites','Full','Premier')
    GROUP BY
        purchaser_ticketing_id
),
attendance_info AS (
    SELECT
        ticketing_id,
        num_games_attended,
        self_show_rate,
        seat_show_rate,
        real_show_rate
    FROM
        custom.cth_v_show_rates
    WHERE
        season = '2023-24'
),
total_spend AS (
    SELECT
        purchaser_ticketing_id,
        SUM(gross_revenue) AS gross_revenue
    FROM
        custom.cth_v_historical_ticket
    WHERE
        season = '2023-24'
    GROUP BY
        purchaser_ticketing_id
),
location_ticket_type AS (
    SELECT
        purchaser_ticketing_id,
        LISTAGG(DISTINCT ticket_type, ', ') AS ticket_types,
        LISTAGG(DISTINCT arena_level_internal, ', ') AS arena_level_internals
    FROM
        custom.cth_v_historical_ticket
    WHERE
        ticket_type IN ('Annual Suites','Full','Premier')
        AND season = '2023-24'
    GROUP BY
        purchaser_ticketing_id
),
did_renew AS (
    SELECT
        subs_2324.purch_client_crm_id,
        CASE
            WHEN subs_2425.purch_client_crm_id IS NULL THEN 0
            ELSE 1
        END AS did_renew
    FROM
        custom.cth_v_ticket_subscription_2324 subs_2324
    LEFT JOIN
        custom.cth_v_ticket_subscription_2425 subs_2425 ON subs_2324.purch_client_crm_id = subs_2425.purch_client_crm_id
    GROUP BY
        subs_2425.purch_client_crm_id,
        subs_2324.purch_client_crm_id)
SELECT
    zip.purch_client_crm_id,
    zip.zip_code,
    touchpoints_agg.touchpoints,
    tenure.tenure,
    attendance_info.real_show_rate,
    attendance_info.seat_show_rate,
    attendance_info.self_show_rate,
    attendance_info.num_games_attended,
    total_spend.gross_revenue,
    location_ticket_type.ticket_types,
    location_ticket_type.arena_level_internals,
    did_renew.did_renew
FROM
    zip
LEFT JOIN
    touchpoints_agg ON zip.purch_client_crm_id = touchpoints_agg.primary_ticketing_id
LEFT JOIN
    tenure ON zip.purch_client_crm_id = tenure.purchaser_ticketing_id
LEFT JOIN
    attendance_info ON zip.purch_client_crm_id = attendance_info.ticketing_id
LEFT JOIN
    total_spend ON zip.purch_client_crm_id = total_spend.purchaser_ticketing_id
LEFT JOIN
    location_ticket_type ON zip.purch_client_crm_id = location_ticket_type.purchaser_ticketing_id
LEFT JOIN
    did_renew on zip.purch_client_crm_id = did_renew.purch_client_crm_id
"""

df_2324 = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [8]:
df_2324

,purch_client_crm_id,zip_code,touchpoints,tenure,real_show_rate,seat_show_rate,self_show_rate,num_games_attended,gross_revenue,ticket_types,arena_level_internals,did_renew
0,23448985,None,NaN,1,0.313433,0.435294,0.247059,18,378.78,Full,Lowers,1.0
1,15915892,33428,23.0,1,0.968750,0.976190,0.500000,21,9238.38,Full,Lowers,1.0
2,21456873,33324,4.0,1,0.937500,0.928571,0.714286,30,7571.32,Full,Lowers,1.0
3,12290968,33076,7.0,1,1.000000,1.000000,0.976190,41,4171.93,Full,Uppers,1.0
4,13962710,33351,22.0,1,0.900000,0.884956,0.672566,29,10565.94,Full,Clubs,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4205,24441384,33162,2.0,1,0.575758,0.908730,0.075397,5,23792.52,Full,Lowers,1.0
4206,21474048,33437,2.0,1,0.927835,0.940476,0.535714,25,18298.07,Full,Uppers,1.0
4207,8906727,33019,15.0,1,0.781250,0.865385,0.480769,13,17286.92,Premier,Lowers,1.0
4208,16091918,33064,NaN,1,0.964602,0.958824,0.629412,34,31314.62,Full,Lowers,1.0
